In [9]:
'''
Girish Rajani-Bathija
A20503736
CS 577 - F22
Assignment 4 Binary Classification
'''

from keras import models, layers, optimizers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, shutil

In [25]:
#Load train and test data as dataframes 
train_dataset = pd.read_csv('./dataset/train.csv')
test_dataset = pd.read_csv('./dataset/test.csv')

print("Length of training data is: ", train_dataset.shape[0])
print("Length of testing data is: ", test_dataset.shape[0])

print("\nSummary of first 10 rows of train data:")
train_dataset.head(10)


Length of training data is:  3662
Length of testing data is:  1928

Summary of first 10 rows of train data:


,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0
5,0083ee8054ee,4
6,0097f532ac9f,0
7,00a8624548a9,2
8,00b74780d31d,2
9,00cb6555d108,1


In [47]:
#Split the training data into the 5 classes
class_0=[]
class_1=[]
class_2=[]
class_3=[]
class_4=[]

for diag0 in train_dataset[train_dataset['diagnosis']==0]['id_code']:
    class_0.append(diag0)
for diag1 in train_dataset[train_dataset['diagnosis']==1]['id_code']:
    class_1.append(diag1)
for diag2 in train_dataset[train_dataset['diagnosis']==2]['id_code']:
    class_2.append(diag2)
for diag3 in train_dataset[train_dataset['diagnosis']==3]['id_code']:
    class_3.append(diag3)
for diag4 in train_dataset[train_dataset['diagnosis']==4]['id_code']:
    class_4.append(diag4)

'''
Note, if we wanna create validation sets then create 2 lists (class_0_train and class_0_val) 
and do a split on class_0 list from above
'''

#add .png extension to the end of each id which will be used later
for i in range(0, len(class_0)):
    class_0[i] = class_0[i]+".png"

for i in range(0, len(class_1)):
    class_1[i] = class_1[i]+".png"

for i in range(0, len(class_2)):
    class_2[i] = class_2[i]+".png"

for i in range(0, len(class_3)):
    class_3[i] = class_3[i]+".png"

for i in range(0, len(class_4)):
    class_4[i] = class_4[i]+".png"

print("Length of training data in class 0:", len(class_0))
print("Length of training data in class 1:", len(class_1))
print("Length of training data in class 2:", len(class_2))
print("Length of training data in class 3:", len(class_3))
print("Length of training data in class 4:", len(class_4))

class_0

Length of training data in class 0: 1805
Length of training data in class 1: 370
Length of training data in class 2: 999
Length of training data in class 3: 193
Length of training data in class 4: 295


['002c21358ce6.png',
 '005b95c28852.png',
 '0097f532ac9f.png',
 '00cc2b75cddd.png',
 '00f6c1be5a33.png',
 '0125fbd2e791.png',
 '014508ccb9cb.png',
 '0151781fe50b.png',
 '0182152c50de.png',
 '01d9477b1171.png',
 '01f7bb8be950.png',
 '0212dd31f623.png',
 '022f820027b8.png',
 '0231642cf1c2.png',
 '0232dfea7547.png',
 '02358b47ea89.png',
 '02cd34a85b24.png',
 '02da652c74b8.png',
 '0304bedad8fe.png',
 '03b373718013.png',
 '0423237770a7.png',
 '04579e31e4be.png',
 '04aef84a2cc1.png',
 '04efb1a284cc.png',
 '050bb1eafa76.png',
 '05113073b268.png',
 '05195a3db5e2.png',
 '05339950962e.png',
 '054b1b305160.png',
 '0551676cc2aa.png',
 '05b1bb2bdb81.png',
 '05e9126dfa5c.png',
 '060e00d1e2ab.png',
 '06586082a24d.png',
 '06be1092a062.png',
 '0709652336e2.png',
 '070d4ce5fd90.png',
 '07596907347b.png',
 '0773a1c326ad.png',
 '07751b94a88a.png',
 '0790515cf5af.png',
 '07a0be6b347f.png',
 '07a2b8cabf6b.png',
 '07e827469099.png',
 '07f5d7baf907.png',
 '08037e4490e5.png',
 '084c02cf077f.png',
 '0851d6a6958

In [30]:
#Path to directory where the images are
train_dataset_dir = './dataset/train_images/'
test_dataset_dir = './dataset/test_images/'

#Directory where the new labelled training dataset will be created and stored
new_dataset = './new_dataset'
os.mkdir(new_dataset)
train_dir = os.path.join(new_dataset, 'train')
os.mkdir(train_dir)

#Directories with training images for each class
train_class0 = os.path.join(train_dir, 'class0')
os.mkdir(train_class0)
train_class1 = os.path.join(train_dir, 'class1')
os.mkdir(train_class1)
train_class2 = os.path.join(train_dir, 'class2')
os.mkdir(train_class2)
train_class3 = os.path.join(train_dir, 'class3')
os.mkdir(train_class3)
train_class4 = os.path.join(train_dir, 'class4')
os.mkdir(train_class4)


In [51]:
#Use labels from class list to add the images to their appropriate class directory

#Class 0:
for k in class_0:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class0, k)
    shutil.copyfile(src, dst)

#Class 1:
for k in class_1:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class1, k)
    shutil.copyfile(src, dst)

#Class 2:
for k in class_2:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class2, k)
    shutil.copyfile(src, dst)

#Class 3:
for k in class_3:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class3, k)
    shutil.copyfile(src, dst)

#Class 4:
for k in class_4:
    src = os.path.join(train_dataset_dir, k)
    dst = os.path.join(train_class4, k)
    shutil.copyfile(src, dst)
print("done")

done


In [52]:
#Build model with convolution layers, pooling, and normalization
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(5, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 74, 74, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                        